In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')


In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
df['cough'].unique()

array(['Mild', 'Strong'], dtype=object)

In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split


In [18]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [21]:
X_train.sample(5)

,age,gender,fever,cough,city
82,24,Male,98.0,Mild,Kolkata
10,75,Female,NaN,Mild,Delhi
73,34,Male,98.0,Strong,Kolkata
75,5,Male,102.0,Mild,Kolkata
35,82,Female,102.0,Strong,Bangalore


In [22]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

In [23]:
X_train_fever.shape

(80, 1)

In [24]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])

In [25]:
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [27]:
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']]).toarray()
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']]).toarray()
X_train_gender_city.shape

(80, 4)

In [28]:
X_train_gender_city


array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],


In [29]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

In [30]:
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [32]:
X_train_age.shape

(80, 1)

In [33]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [34]:
from sklearn.compose import ColumnTransformer

In [35]:
transformer = ColumnTransformer(transformers=
[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [36]:
transformer.fit_transform(X_train)

array([[101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  19.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  50.        ],
       [ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  80.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [102.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.    

In [37]:
transformer.fit_transform(X_train).shape

(80, 7)

In [39]:
transformer.transform(X_test)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  11.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [100.82191781,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [100.82191781,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  59.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.    

In [38]:
transformer.transform(X_test).shape


(20, 7)